In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Resolution Hours Ensemble Model - Improved
# 
# This notebook creates a more realistic ensemble model to predict total resolution hours for Jira issues.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
import pickle
import os
from scipy.stats import spearmanr

# Set random seed for reproducibility
np.random.seed(42)

# Load the dataset
data_path = 'merged_task_data/merged_project_task_data.csv'
df = pd.read_csv(data_path, low_memory=False)
print(f"Original dataset shape: {df.shape}")

# Convert string columns that should be numeric
for col in df.columns:
    if col.startswith('fields.issuetype') or col.startswith('fields.priority'):
        # Keep these as string/categorical
        continue
    try:
        # Try converting to numeric, coerce errors to NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
    except:
        # If conversion fails completely, leave as is
        pass

# Use only 10% of the data for faster processing
df = df.sample(frac=0.05, random_state=42)
print(f"Reduced dataset shape (10%): {df.shape}")

# Define target variable - to be used only as the target, never as a feature
target_variable = 'total_resolution_hours'

# Define fields that must be excluded
fields_to_exclude = ['fields.status.name', 'fields.project.key', 'fields.priority.name', 'fields.project.name']

# Check which of these fields actually exist in the dataframe
existing_fields_to_exclude = [field for field in fields_to_exclude if field in df.columns]
print(f"\nFields to exclude that exist in the dataframe: {existing_fields_to_exclude}")

# Remove the fields from the dataframe itself to avoid correlation errors
if existing_fields_to_exclude:
    print(f"Dropping these fields from the dataframe: {existing_fields_to_exclude}")
    df = df.drop(columns=existing_fields_to_exclude)
    print(f"Dataframe shape after dropping: {df.shape}")

# Check for missing values in the target variable
print(f"Missing values in target: {df[target_variable].isna().sum()}")

# Drop rows with missing target values
df = df.dropna(subset=[target_variable])
print(f"Dataset shape after dropping missing targets: {df.shape}")

# Plot distribution of target variable
plt.figure(figsize=(10, 6))
sns.histplot(df[target_variable].clip(0, 500), bins=50)
plt.title('Distribution of Total Resolution Hours (clipped at 500 for visibility)')
plt.xlabel('Hours')
plt.ylabel('Count')
os.makedirs('task_estimation_results_improved', exist_ok=True)
plt.savefig('task_estimation_results_improved/target_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# Identify numeric columns for correlation analysis
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
# Exclude target from potential features
numeric_features = [col for col in numeric_columns if col != target_variable]
print(f"\nUsing {len(numeric_features)} numeric features for correlation calculation")

# Calculate correlations between features and target
if target_variable in df.columns and numeric_features:
    # Create a dataframe with just the features and target for correlation
    corr_df = df[numeric_features + [target_variable]]
    target_correlations = corr_df.corr()[target_variable].drop(target_variable).sort_values(ascending=False)
    
    print("Top 10 correlated features with target:")
    print(target_correlations.head(10))
    print("\nBottom 10 correlated features with target:")
    print(target_correlations.tail(10))
    
    # Print all features that have correlation > 0.8 with target
    high_corr_features = target_correlations[target_correlations > 0.8].index.tolist()
    print(f"\nFeatures with >0.8 correlation with target: {high_corr_features}")
else:
    print("Cannot calculate target correlations - either target or features missing")
    high_corr_features = []
    
# Define suspicious features (features that may lead to data leakage)
suspicious_features = [
    # Resolution-related features that would leak the target information
    'avg_resolution_hours', 'median_resolution_hours', 
    'resolution_hours', 'log_resolution_hours'
]

# Add highly correlated features to suspicious list
suspicious_features.extend(high_corr_features)

# Add excluded fields to suspicious list
suspicious_features.extend(fields_to_exclude)


# Make sure target variable is not in suspicious features list (it's our target, not a feature)
if target_variable in suspicious_features:
    suspicious_features.remove(target_variable)
    
print(f"\nRemoving suspicious features: {suspicious_features}")

# Original features list, excluding suspicious features
features = [
    'fields.issuetype.id', 'fields.priority.id', 'priority_id', 'issue_type_id',
    'type_task', 'type_bug', 'inward_count', 'outward_count', 'type_sub_task',
    'created_hour', 'created_month', 'created_year',
    'is_type_bug', 'is_type_task', 'is_type_story', 'is_type_improvement',
    'is_type_new_feature', 'is_type_epic', 'is_type_sub-task',
    'is_priority_blocker', 'is_priority_critical', 'is_priority_major',
    'is_priority_minor', 'is_priority_trivial',
    'pct_minmax__type_bug_pct', 'pct_minmax__type_task_pct',
    'pct_minmax__type_new_feature_pct', 'pct_minmax__type_epic_pct',
    'pct_minmax__type_improvement_pct', 'pct_minmax__type_story_pct',
    'pct_minmax__type_documentation_pct', 'pct_minmax__priority_critical_pct',
    'pct_minmax__priority_blocker_pct', 'pct_minmax__priority_high_pct',
    'pct_minmax__priority_low_pct',
    'remainder__team_size_creators', 'remainder__team_size_assignees',
    'remainder__team_size_combined',
    'stat_robust__weighted_priority_score', 'stat_robust__issue_type_entropy',
    'stat_robust__high_to_low_priority_ratio', 'stat_robust__bug_ratio'
]

# Ensure we exclude the additional fields by checking if they exist in the dataset
for field in fields_to_exclude:
    if field in df.columns and field not in suspicious_features:
        suspicious_features.append(field)
        print(f"Added {field} to suspicious features list")

# Filter dataset to include only features available in our dataset and not in suspicious list
available_features = [f for f in features if f in df.columns and f not in suspicious_features]
missing_features = [f for f in features if f not in df.columns]

print(f"Available features: {len(available_features)}")
print(f"Missing features: {len(missing_features)}")
if missing_features:
    print(f"Missing feature list: {missing_features}")
    
# Make absolutely sure we're not using the target as a feature
if target_variable in available_features:
    available_features.remove(target_variable)
    print(f"WARNING: Removed target variable '{target_variable}' from features list")
    
# Explicitly check all fields that need to be dropped
print("\nChecking for fields that need to be dropped:")
for field in fields_to_exclude:
    if field in df.columns:
        print(f"Field '{field}' exists in the dataframe")
        if field in available_features:
            available_features.remove(field)
            print(f"-> Removed '{field}' from available features")
    else:
        print(f"Field '{field}' does not exist in the dataframe")

# Double-check if any of these fields might be in the final feature list
for field in fields_to_exclude:
    if field in available_features:
        available_features.remove(field)
        print(f"-> Explicitly removed '{field}' from available features")
        
# Print final confirmation
print("\nConfirming excluded fields:")
for field in fields_to_exclude:
    print(f"'{field}' is {'NOT' if field not in available_features else 'STILL'} in the feature list")

# Check for multicollinearity
# Use only numeric features for correlation calculation
numeric_available_features = [f for f in available_features if f in numeric_features]
print(f"\nUsing {len(numeric_available_features)} numeric features for multicollinearity check")

if len(numeric_available_features) > 0:
    # Verify one more time we don't have the target as a feature
    if target_variable in numeric_available_features:
        numeric_available_features.remove(target_variable)
        print(f"WARNING: Removed target variable from multicollinearity calculation")
    
    correlation_matrix = df[numeric_available_features].corr().abs()
    upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool_))
    high_corr_pairs = [(col1, col2) for col1 in upper_tri.columns for col2 in upper_tri.index if upper_tri.loc[col2, col1] > 0.95]

    print(f"\nHigh correlation pairs (>0.95):")
    for col1, col2 in high_corr_pairs:
        print(f"{col1} - {col2}: {upper_tri.loc[col2, col1]:.4f}")
    
    # Optionally remove one of each highly correlated pair
    features_to_drop = []
    for col1, col2 in high_corr_pairs:
        # Keep the first one, drop the second one
        if col2 not in features_to_drop:
            features_to_drop.append(col2)

    print(f"\nDropping features due to multicollinearity: {features_to_drop}")
else:
    print("No numeric features available for multicollinearity check")
    features_to_drop = []

final_features = [f for f in available_features if f not in features_to_drop]
print(f"Final feature count: {len(final_features)}")

# Final check to ensure we've dropped the specified fields
for field in fields_to_exclude:
    if field in final_features:
        final_features.remove(field)
        print(f"Final check: Removed '{field}' from final_features")
        
# Print confirmation of fields not in final features
print("\nFinal confirmation - excluded fields:")
for field in fields_to_exclude:
    present = field in final_features
    print(f"'{field}' is {'STILL in' if present else 'NOT in'} the final feature list")

# Prepare the data
X = df[final_features].copy()
y = df[target_variable]  # The target variable

# Fill any remaining NaN values with median
for col in X.columns:
    X[col] = X[col].fillna(X[col].median())

# Split the data with stratification on binned target to ensure similar distributions
y_binned = pd.qcut(y, q=10, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y_binned)

# Final verification: ensure target is not used as a feature
print("\nFinal verification before model training:")
print(f"Target variable: {target_variable}")
print(f"Number of features: {len(final_features)}")
if target_variable in final_features:
    print(f"ERROR: Target variable found in feature list!")
    final_features.remove(target_variable)
    X = df[final_features].copy()  # Recreate X without the target
    print(f"Removed target from features. New feature count: {len(final_features)}")
    
    # Re-split with corrected features
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y_binned)
else:
    print("Verified: Target is not being used as a feature")

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save data splits for later reference
with open('task_estimation_results_improved/data_splits.pkl', 'wb') as f:
    pickle.dump({
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'feature_names': final_features,
        'scaler': scaler
    }, f)

# Define the models with more conservative hyperparameters to prevent overfitting
models = {
    'Random_Forest': RandomForestRegressor(
        n_estimators=100, 
        max_depth=10,
        min_samples_leaf=5,
        max_features='sqrt',
        random_state=42
    ),
    'Gradient_Boosting': GradientBoostingRegressor(
        n_estimators=100, 
        max_depth=5,
        learning_rate=0.05,
        min_samples_leaf=5,
        random_state=42
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=100, 
        max_depth=5, 
        learning_rate=0.05,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
}

# Perform cross-validation to get more reliable performance estimates
print("Performing 5-fold cross-validation...")
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    print(f"{name} CV R² scores: {cv_scores}")
    print(f"{name} CV R² mean: {cv_scores.mean():.4f}, std: {cv_scores.std():.4f}")

# Train and evaluate each model
results = {}
predictions = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    predictions[name] = y_pred
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    spearman_corr, _ = spearmanr(y_test, y_pred)  # Rank correlation, less affected by outliers
    
    results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Spearman': spearman_corr
    }
    
    print(f"{name} - MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}, Spearman: {spearman_corr:.4f}")
    
    # Save the model
    with open(f'task_estimation_results_improved/{name}_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    # Plot actual vs predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([0, y_test.max()], [0, y_test.max()], 'r--')
    plt.xlabel('Actual Hours')
    plt.ylabel('Predicted Hours')
    plt.title(f'{name} - Actual vs Predicted')
    plt.savefig(f'task_estimation_results_improved/{name}_predictions.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Add residual subplot
    plt.figure(figsize=(10, 6))
    residuals = y_test - y_pred
    plt.scatter(y_pred, residuals, alpha=0.3)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted Hours')
    plt.ylabel('Residuals (Actual - Predicted)')
    plt.title(f'{name} - Residual Plot')
    plt.savefig(f'task_estimation_results_improved/{name}_residuals.png', dpi=300, bbox_inches='tight')
    plt.close()

# Create an ensemble prediction by averaging the predictions
ensemble_pred = np.mean([predictions[name] for name in models.keys()], axis=0)

# Calculate metrics for the ensemble
ensemble_mae = mean_absolute_error(y_test, ensemble_pred)
ensemble_rmse = np.sqrt(mean_squared_error(y_test, ensemble_pred))
ensemble_r2 = r2_score(y_test, ensemble_pred)
ensemble_spearman, _ = spearmanr(y_test, ensemble_pred)

print(f"Ensemble - MAE: {ensemble_mae:.2f}, RMSE: {ensemble_rmse:.2f}, R2: {ensemble_r2:.4f}, Spearman: {ensemble_spearman:.4f}")

# Plot actual vs predicted for ensemble
plt.figure(figsize=(10, 6))
plt.scatter(y_test, ensemble_pred, alpha=0.3)
plt.plot([0, y_test.max()], [0, y_test.max()], 'r--')
plt.xlabel('Actual Hours')
plt.ylabel('Predicted Hours')
plt.title('Ensemble - Actual vs Predicted')
plt.savefig('task_estimation_results_improved/Ensemble_predictions.png', dpi=300, bbox_inches='tight')
plt.close()

# Plot residuals for ensemble
plt.figure(figsize=(10, 6))
ensemble_residuals = y_test - ensemble_pred
plt.scatter(ensemble_pred, ensemble_residuals, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Hours')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Ensemble - Residual Plot')
plt.savefig('task_estimation_results_improved/Ensemble_residuals.png', dpi=300, bbox_inches='tight')
plt.close()

# Identify the best model
best_model_name = min(results, key=lambda x: results[x]['MAE'])
best_model = models[best_model_name]
print(f"Best model based on MAE: {best_model_name}")

# Save the best model as the one to use for predictions
with open('task_estimation_results_improved/best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Plot feature importance for the best model
if hasattr(best_model, 'feature_importances_'):
    # Get feature importances
    importances = best_model.feature_importances_
    
    # Sort features by importance
    indices = np.argsort(importances)[::-1]
    
    # Plot the top 20 features
    plt.figure(figsize=(12, 8))
    plt.title(f'Feature Importance - {best_model_name}')
    plt.bar(range(min(20, len(final_features))), 
            importances[indices[:20]], 
            align='center')
    plt.xticks(range(min(20, len(final_features))), 
               [final_features[i] for i in indices[:20]], 
               rotation=90)
    plt.tight_layout()
    plt.savefig('task_estimation_results_improved/best_model_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a DataFrame for easier analysis
    feature_importance_df = pd.DataFrame({
        'Feature': final_features,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    print("Top 10 most important features:")
    print(feature_importance_df.head(10))

    # Calculate and plot correlations between top features and the target
    top_features = [f for f in feature_importance_df['Feature'].head(10).tolist() if f in numeric_features]
    if top_features:
        # Final check that we don't include the target in the correlation matrix
        if target_variable in top_features:
            top_features.remove(target_variable)
            print(f"WARNING: Removed target from top features correlation")
            
        print(f"Calculating correlations for top {len(top_features)} numeric features")
        if top_features:
            corr_columns = top_features + [target_variable]
            corr_data = df[corr_columns].copy()
            
            plt.figure(figsize=(12, 10))
            sns.heatmap(corr_data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
            plt.title('Correlation Between Top Features and Target')
            plt.tight_layout()
            plt.savefig('task_estimation_results_improved/top_features_correlation.png', dpi=300, bbox_inches='tight')
            plt.close()
        else:
            print("No valid top features remain for correlation analysis")
    else:
        print("No numeric top features available for correlation analysis")

    # Group feature importance by categories
    # Define categories based on feature name prefixes
    categories = {
        'Issue Type': [f for f in final_features if 'type_' in f or 'is_type_' in f],
        'Priority': [f for f in final_features if 'priority' in f],
        'Project Stats': [f for f in final_features if 'count_' in f or 'pct_' in f],
        'Team Size': [f for f in final_features if 'team_size' in f],
        'Created Time': [f for f in final_features if 'created_' in f],
        'Robust Stats': [f for f in final_features if 'stat_robust' in f],
        'Other': []
    }
    
    # Assign remaining features to 'Other'
    for feature in final_features:
        if not any(feature in cat_features for cat_features in categories.values()):
            categories['Other'].append(feature)
    
    # Calculate importance by category
    category_importance = {}
    for category, cat_features in categories.items():
        if cat_features:  # Skip empty categories
            total_importance = sum(feature_importance_df.loc[feature_importance_df['Feature'].isin(cat_features), 'Importance'])
            category_importance[category] = total_importance
    
    # Plot category importance
    plt.figure(figsize=(10, 6))
    categories_sorted = sorted(category_importance.items(), key=lambda x: x[1], reverse=True)
    categories_names = [item[0] for item in categories_sorted]
    categories_values = [item[1] for item in categories_sorted]
    
    plt.bar(categories_names, categories_values)
    plt.xlabel('Feature Category')
    plt.ylabel('Total Importance')
    plt.title('Feature Importance by Category')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('task_estimation_results_improved/best_model_feature_importance_by_category.png', dpi=300, bbox_inches='tight')
    plt.close()

# Plot actual vs predicted in task_results folder for overall project view
os.makedirs('task_results_improved', exist_ok=True)

plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions[best_model_name], alpha=0.3)
plt.plot([0, y_test.max()], [0, y_test.max()], 'r--')
plt.xlabel('Actual Hours')
plt.ylabel('Predicted Hours')
plt.title(f'Best Model ({best_model_name}) - Actual vs Predicted')
plt.savefig('task_results_improved/actual_vs_predicted.png', dpi=300, bbox_inches='tight')
plt.close()

# Plot feature importance in task_results folder
if hasattr(best_model, 'feature_importances_'):
    plt.figure(figsize=(12, 8))
    plt.title(f'Feature Importance - {best_model_name}')
    plt.bar(range(min(20, len(final_features))), 
            importances[indices[:20]], 
            align='center')
    plt.xticks(range(min(20, len(final_features))), 
               [final_features[i] for i in indices[:20]], 
               rotation=90)
    plt.tight_layout()
    plt.savefig('task_results_improved/task_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()

print("\nModel Summary:")
print("==============")
print("We've created an improved ensemble model to predict resolution hours for Jira issues. The improvements include:")
print("1. Removing features with potential data leakage (high correlation with target)")
print("2. Addressing multicollinearity by removing highly correlated features")
print("3. Using cross-validation to get more reliable performance estimates")
print("4. Applying more conservative hyperparameters to prevent overfitting")
print("5. Adding residual plots to better understand model errors")
print("6. Including Spearman rank correlation as a metric less affected by outliers")
print("7. Removing project and status specific features to prevent overfitting to specific projects")
print("\nThis improved model provides more realistic estimates and better generalization to new data.")

Original dataset shape: (1575833, 63)
Reduced dataset shape (10%): (78792, 63)

Fields to exclude that exist in the dataframe: ['fields.status.name', 'fields.project.key', 'fields.priority.name', 'fields.project.name']
Dropping these fields from the dataframe: ['fields.status.name', 'fields.project.key', 'fields.priority.name', 'fields.project.name']
Dataframe shape after dropping: (78792, 59)
Missing values in target: 0
Dataset shape after dropping missing targets: (78792, 59)

Using 57 numeric features for correlation calculation
Top 10 correlated features with target:
count_std__total_issues                 0.862916
remainder__team_size_creators           0.756246
remainder__team_size_combined           0.756246
time_power__project_duration_days       0.436095
stat_robust__bug_ratio                  0.295816
pct_minmax__type_bug_pct                0.264590
avg_resolution_hours                    0.258526
stat_robust__weighted_priority_score    0.187998
age_days                      